In [7]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)


features =20
filename="./dataset/20200604_v1.csv"

# 데이터 읽어 들이기--- (※1)
#ci_year	ci_term	si_grad	si_ent_gubun	agescale	si_sex	si_last_hak	schoarYn	
#accept_point	avg_logincnt	avg_att_time	avg_att_rate	avg_bbscnt	avg_startlogin	entYn	classcnt	
#avg_te_avg	avg_te_passrate	gradcnt	counsel	stdResult

df_pre=pd.read_csv(filename,encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용
#df_pre=pd.read_csv('./dataset/20200612_app_point.csv',encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용

#df_pre_new=df_pre[df_pre.stdResult!=3] #졸업생은 제외



In [8]:

df_pre


,ci_year,ci_term,si_grad,si_ent_gubun,agescale,si_sex,si_last_hak,schoarYn,accept_point,avg_logincnt,...,avg_att_rate,avg_bbscnt,avg_startlogin,entYn,classcnt,pre_avg_te_avg,pre_avg_te_passrate,pre_gradcnt,counsel,stdResult
0,2015,1,4,1,60,1,1,1,0,87,...,99.56,0,38,0,5,3.50,100,7,0,1
1,2015,1,4,1,30,2,1,1,0,26,...,85.52,0,18,0,4,2.00,100,10,0,1
2,2015,1,4,1,40,2,2,0,0,26,...,100.00,0,13,0,3,2.50,100,10,0,1
3,2015,1,4,1,20,1,2,0,0,8,...,84.21,0,5,0,6,2.00,100,8,0,1
4,2015,1,4,1,30,1,2,0,0,5,...,81.58,0,4,0,1,0.00,0,12,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58083,2019,2,4,2,50,1,3,1,70,74,...,100.00,0,30,0,6,3.42,100,1,0,0
58084,2019,2,4,2,50,2,3,1,70,50,...,99.72,0,20,0,8,3.67,100,1,0,0
58085,2019,2,4,2,40,2,3,1,70,63,...,100.00,6,22,0,6,3.33,100,1,0,0
58086,2019,2,4,2,40,2,2,1,70,98,...,100.00,12,47,0,8,3.67,100,1,0,0


In [9]:
# 데이터 내부의 기호를 숫자로 변환하기--- (※2)
df = df_pre.sample(frac=1)

dataset = df.values
X = dataset[:,0:features]
Y = dataset[:,features]

# 학습 전용과 테스트 전용 데이터로 나누기 --- (※3)
#테스트, 검증데이터 분할 7:3 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

#데이터 정규화
X_train, X_test = X_train / 255, X_test / 255

from keras.layers import Dense, Dropout, Activation
# 모델 설정
model = Sequential()
model.add(Dense(28, input_dim=features, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(15, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일
#model.compile(loss='binary_crossentropy',
model.compile(loss='mean_squared_error',
           optimizer='adam',
           # metrics=['accuracy'])
           metrics=['accuracy',recall_m,precision_m,f1_m])

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

# 모델 실행 및 저장
history = model.fit(X_train, Y_train, validation_split=0.20, epochs=500, batch_size=100, 
                    callbacks=[early_stopping_callback])


# 결과 출력
print("\n epoches-500, bat_size=100")
print("\n 학습중단 + 모델 성능개선 : arly_stopping_callback:")
print("\n 예측정확도: %.4f" % (model.evaluate(X_test, Y_test)[1]))


#테스트 데이터 검증
loss, accuracy, recall,precision,f1_socre  = model.evaluate(X_test, Y_test)
#accuracy  = model.evaluate(X_test, Y_test)

print("\n accuracy, precision, recall, f1_score")
print("\n  %.4f, %.4f, %4f, %.4f" % (accuracy, precision ,recall, f1_socre))

#print("\n accuracy")
#print("\n  %.4f % (accuracy)")

Train on 32528 samples, validate on 8133 samples
Epoch 1/500
32528/32528 [==============================] - 1s 29us/step - loss: 0.2576 - accuracy: 0.6384 - recall_m: 0.8414 - precision_m: 0.7038 - f1_m: 0.7568 - val_loss: 0.2129 - val_accuracy: 0.7001 - val_recall_m: 1.0000 - val_precision_m: 0.7003 - val_f1_m: 0.8230
Epoch 2/500
32528/32528 [==============================] - 1s 19us/step - loss: 0.2152 - accuracy: 0.7023 - recall_m: 0.9942 - precision_m: 0.7040 - f1_m: 0.8234 - val_loss: 0.2108 - val_accuracy: 0.7001 - val_recall_m: 1.0000 - val_precision_m: 0.7003 - val_f1_m: 0.8230
Epoch 3/500
32528/32528 [==============================] - 1s 18us/step - loss: 0.2116 - accuracy: 0.7034 - recall_m: 0.9995 - precision_m: 0.7035 - f1_m: 0.8249 - val_loss: 0.2096 - val_accuracy: 0.7001 - val_recall_m: 1.0000 - val_precision_m: 0.7003 - val_f1_m: 0.8230
Epoch 4/500
32528/32528 [==============================] - 1s 17us/step - loss: 0.2101 - accuracy: 0.7035 - recall_m: 0.9999 - precisio

Epoch 31/500
32528/32528 [==============================] - 1s 20us/step - loss: 0.1967 - accuracy: 0.7093 - recall_m: 0.9901 - precision_m: 0.7107 - f1_m: 0.8266 - val_loss: 0.1901 - val_accuracy: 0.7032 - val_recall_m: 0.9968 - val_precision_m: 0.7036 - val_f1_m: 0.8241
Epoch 32/500
32528/32528 [==============================] - 1s 18us/step - loss: 0.1965 - accuracy: 0.7085 - recall_m: 0.9886 - precision_m: 0.7099 - f1_m: 0.8256 - val_loss: 0.1881 - val_accuracy: 0.7028 - val_recall_m: 0.9969 - val_precision_m: 0.7033 - val_f1_m: 0.8240
Epoch 33/500
32528/32528 [==============================] - 1s 18us/step - loss: 0.1944 - accuracy: 0.7084 - recall_m: 0.9888 - precision_m: 0.7101 - f1_m: 0.8258 - val_loss: 0.1858 - val_accuracy: 0.7031 - val_recall_m: 0.9968 - val_precision_m: 0.7036 - val_f1_m: 0.8241
Epoch 34/500
32528/32528 [==============================] - 1s 19us/step - loss: 0.1930 - accuracy: 0.7083 - recall_m: 0.9869 - precision_m: 0.7108 - f1_m: 0.8255 - val_loss: 0.1798

32528/32528 [==============================] - 1s 19us/step - loss: 0.1806 - accuracy: 0.7216 - recall_m: 0.9063 - precision_m: 0.7504 - f1_m: 0.8196 - val_loss: 0.1612 - val_accuracy: 0.7038 - val_recall_m: 0.9964 - val_precision_m: 0.7042 - val_f1_m: 0.8244
Epoch 62/500
32528/32528 [==============================] - 1s 17us/step - loss: 0.1812 - accuracy: 0.7193 - recall_m: 0.8920 - precision_m: 0.7545 - f1_m: 0.8162 - val_loss: 0.1579 - val_accuracy: 0.7086 - val_recall_m: 0.9942 - val_precision_m: 0.7082 - val_f1_m: 0.8264
Epoch 63/500
32528/32528 [==============================] - 1s 19us/step - loss: 0.1814 - accuracy: 0.7211 - recall_m: 0.8936 - precision_m: 0.7556 - f1_m: 0.8176 - val_loss: 0.1576 - val_accuracy: 0.7128 - val_recall_m: 0.9907 - val_precision_m: 0.7123 - val_f1_m: 0.8279
Epoch 64/500
32528/32528 [==============================] - 1s 18us/step - loss: 0.1824 - accuracy: 0.7170 - recall_m: 0.9003 - precision_m: 0.7493 - f1_m: 0.8162 - val_loss: 0.1670 - val_accura

32528/32528 [==============================] - 1s 19us/step - loss: 0.1854 - accuracy: 0.7109 - recall_m: 0.9920 - precision_m: 0.7114 - f1_m: 0.8278 - val_loss: 0.1629 - val_accuracy: 0.7128 - val_recall_m: 0.9912 - val_precision_m: 0.7122 - val_f1_m: 0.8280
Epoch 92/500
32528/32528 [==============================] - 1s 17us/step - loss: 0.1858 - accuracy: 0.7091 - recall_m: 0.9902 - precision_m: 0.7107 - f1_m: 0.8267 - val_loss: 0.1633 - val_accuracy: 0.7107 - val_recall_m: 0.9933 - val_precision_m: 0.7100 - val_f1_m: 0.8273
Epoch 93/500
32528/32528 [==============================] - 1s 17us/step - loss: 0.1830 - accuracy: 0.7108 - recall_m: 0.9923 - precision_m: 0.7110 - f1_m: 0.8276 - val_loss: 0.1613 - val_accuracy: 0.7166 - val_recall_m: 0.9890 - val_precision_m: 0.7156 - val_f1_m: 0.8296
Epoch 94/500
32528/32528 [==============================] - 1s 17us/step - loss: 0.1855 - accuracy: 0.7102 - recall_m: 0.9897 - precision_m: 0.7110 - f1_m: 0.8267 - val_loss: 0.1598 - val_accura

Epoch 121/500
32528/32528 [==============================] - 1s 18us/step - loss: 0.1854 - accuracy: 0.7105 - recall_m: 0.9912 - precision_m: 0.7111 - f1_m: 0.8272 - val_loss: 0.1594 - val_accuracy: 0.7113 - val_recall_m: 0.9931 - val_precision_m: 0.7105 - val_f1_m: 0.8275
Epoch 122/500
32528/32528 [==============================] - 1s 17us/step - loss: 0.1851 - accuracy: 0.7099 - recall_m: 0.9915 - precision_m: 0.7105 - f1_m: 0.8270 - val_loss: 0.1596 - val_accuracy: 0.7113 - val_recall_m: 0.9931 - val_precision_m: 0.7105 - val_f1_m: 0.8275
Epoch 123/500
32528/32528 [==============================] - 1s 19us/step - loss: 0.1848 - accuracy: 0.7095 - recall_m: 0.9903 - precision_m: 0.7108 - f1_m: 0.8268 - val_loss: 0.1565 - val_accuracy: 0.7113 - val_recall_m: 0.9931 - val_precision_m: 0.7105 - val_f1_m: 0.8275
Epoch 124/500
32528/32528 [==============================] - 1s 19us/step - loss: 0.1881 - accuracy: 0.7107 - recall_m: 0.9922 - precision_m: 0.7109 - f1_m: 0.8276 - val_loss: 0.

...

Epoch 360/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1843 - accuracy: 0.7111 - recall_m: 0.9926 - precision_m: 0.7110 - f1_m: 0.8276 - val_loss: 0.1537 - val_accuracy: 0.7166 - val_recall_m: 0.9912 - val_precision_m: 0.7149 - val_f1_m: 0.8299
Epoch 361/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1834 - accuracy: 0.7109 - recall_m: 0.9888 - precision_m: 0.7124 - f1_m: 0.8273 - val_loss: 0.1543 - val_accuracy: 0.7120 - val_recall_m: 0.9933 - val_precision_m: 0.7109 - val_f1_m: 0.8279
Epoch 362/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1832 - accuracy: 0.7118 - recall_m: 0.9906 - precision_m: 0.7125 - f1_m: 0.8281 - val_loss: 0.1567 - val_accuracy: 0.7127 - val_recall_m: 0.9927 - val_precision_m: 0.7115 - val_f1_m: 0.8281
Epoch 363/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1853 - accuracy: 0.7102 - recall_m: 0.9897 - precision_m: 0.7113 - f1_m: 0.8269 - val_loss: 0.

Epoch 390/500
32528/32528 [==============================] - 0s 15us/step - loss: 0.1838 - accuracy: 0.7111 - recall_m: 0.9904 - precision_m: 0.7117 - f1_m: 0.8274 - val_loss: 0.1594 - val_accuracy: 0.7099 - val_recall_m: 0.9950 - val_precision_m: 0.7090 - val_f1_m: 0.8272
Epoch 391/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1830 - accuracy: 0.7108 - recall_m: 0.9916 - precision_m: 0.7115 - f1_m: 0.8276 - val_loss: 0.1551 - val_accuracy: 0.7118 - val_recall_m: 0.9933 - val_precision_m: 0.7108 - val_f1_m: 0.8278
Epoch 392/500
32528/32528 [==============================] - 1s 17us/step - loss: 0.1825 - accuracy: 0.7113 - recall_m: 0.9915 - precision_m: 0.7113 - f1_m: 0.8274 - val_loss: 0.1566 - val_accuracy: 0.7145 - val_recall_m: 0.9921 - val_precision_m: 0.7131 - val_f1_m: 0.8290
Epoch 393/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1829 - accuracy: 0.7115 - recall_m: 0.9901 - precision_m: 0.7123 - f1_m: 0.8277 - val_loss: 0.

Epoch 420/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1833 - accuracy: 0.7113 - recall_m: 0.9902 - precision_m: 0.7122 - f1_m: 0.8275 - val_loss: 0.1613 - val_accuracy: 0.7183 - val_recall_m: 0.9909 - val_precision_m: 0.7163 - val_f1_m: 0.8307
Epoch 421/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1827 - accuracy: 0.7104 - recall_m: 0.9913 - precision_m: 0.7111 - f1_m: 0.8273 - val_loss: 0.1544 - val_accuracy: 0.7123 - val_recall_m: 0.9927 - val_precision_m: 0.7113 - val_f1_m: 0.8280
Epoch 422/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1835 - accuracy: 0.7106 - recall_m: 0.9906 - precision_m: 0.7111 - f1_m: 0.8271 - val_loss: 0.1576 - val_accuracy: 0.7115 - val_recall_m: 0.9943 - val_precision_m: 0.7103 - val_f1_m: 0.8278
Epoch 423/500
32528/32528 [==============================] - ETA: 0s - loss: 0.1838 - accuracy: 0.7113 - recall_m: 0.9929 - precision_m: 0.7111 - f1_m: 0.82 - 0s 14us/step - loss

32528/32528 [==============================] - 0s 13us/step - loss: 0.1826 - accuracy: 0.7112 - recall_m: 0.9872 - precision_m: 0.7127 - f1_m: 0.8268 - val_loss: 0.1572 - val_accuracy: 0.7133 - val_recall_m: 0.9927 - val_precision_m: 0.7120 - val_f1_m: 0.8284
Epoch 450/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1835 - accuracy: 0.7115 - recall_m: 0.9887 - precision_m: 0.7123 - f1_m: 0.8271 - val_loss: 0.1571 - val_accuracy: 0.7124 - val_recall_m: 0.9940 - val_precision_m: 0.7110 - val_f1_m: 0.8282
Epoch 451/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1828 - accuracy: 0.7114 - recall_m: 0.9917 - precision_m: 0.7118 - f1_m: 0.8278 - val_loss: 0.1565 - val_accuracy: 0.7124 - val_recall_m: 0.9927 - val_precision_m: 0.7114 - val_f1_m: 0.8280
Epoch 452/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1820 - accuracy: 0.7127 - recall_m: 0.9895 - precision_m: 0.7132 - f1_m: 0.8281 - val_loss: 0.1536 - val_acc

Epoch 479/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1829 - accuracy: 0.7122 - recall_m: 0.9908 - precision_m: 0.7125 - f1_m: 0.8281 - val_loss: 0.1535 - val_accuracy: 0.7178 - val_recall_m: 0.9909 - val_precision_m: 0.7160 - val_f1_m: 0.8304
Epoch 480/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1828 - accuracy: 0.7113 - recall_m: 0.9916 - precision_m: 0.7117 - f1_m: 0.8278 - val_loss: 0.1527 - val_accuracy: 0.7124 - val_recall_m: 0.9927 - val_precision_m: 0.7114 - val_f1_m: 0.8280
Epoch 481/500
32528/32528 [==============================] - 0s 13us/step - loss: 0.1833 - accuracy: 0.7118 - recall_m: 0.9912 - precision_m: 0.7122 - f1_m: 0.8280 - val_loss: 0.1541 - val_accuracy: 0.7152 - val_recall_m: 0.9921 - val_precision_m: 0.7137 - val_f1_m: 0.8293
Epoch 482/500
32528/32528 [==============================] - 0s 14us/step - loss: 0.1833 - accuracy: 0.7117 - recall_m: 0.9901 - precision_m: 0.7123 - f1_m: 0.8277 - val_loss: 0.